In [1]:
from re import sub
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

## Dataset import

In [2]:
df = pd.read_csv(
    "dataset.csv",
    parse_dates=[
        "created_at",
        "Customer Since"
        # "Working Date"
    ],
    na_values=["\\N"],
    usecols={
        "increment_id",
        # "item_id",
        "status",
        "created_at",
        # "sku",
        "price",
        "qty_ordered",
        "grand_total",
        "category_name_1",
        "discount_amount",
        "payment_method",
        # "Working Date",
        # "bi_status",
        # " MV ",
        "Customer Since",
        "Customer ID",
        "Year",
        "Month"
    },
    dtype={
        "increment_id": "category",
        "status": "category",
        # "price": "float64",
        # "qty_ordered": "int64",
        # "grand_total": "float64",
        "category_name_1": "category",
        # "discount_amount": "float64",
        "payment_method": "category",
        # "bi_status": "category",
        # " MV ": "float64",
        "Customer ID": "category",
        "Year": "category",
        "Month": "category"
    },
    low_memory=False
)

FileNotFoundError: [Errno 2] No such file or directory: 'dataset.csv'

In [ ]:
df.dtypes

## Data preprocessing

#### Format and rename column headers

In [ ]:
df.columns = df.columns.map(
    lambda s: '_'.join(sub('([A-Z][a-z]+)', r' \1', sub('([A-Z]+)', r' \1', s.replace('-', ' '))).split()).lower())
df.rename(columns={
    "category_name_1": "category",
    "qty_ordered": "quantity"
}, inplace=True)

#### Handle null values

In [ ]:
df.dropna(how='all', inplace=True)  # Drop all-empty rows
print(len(df.index))
df.dropna(subset=['customer_id', 'status'], inplace=True)
df["category"].fillna("Others", inplace=True)

#### Clean up status category

In [ ]:
df["status"] = df["status"].replace(
    ['complete', 'closed', 'received', 'cod', 'paid', 'exchange', 'payment_review', 'pending', 'processing', 'holded',
     'pending_paypal'],
    "completed"
).replace(
    ['order_refunded', 'refund'],
    "refunded"
).replace(
    ['canceled', 'fraud', np.nan],
    "canceled"
)

#### Clean up payment method category

In [ ]:
df["payment_method"] = df["payment_method"].replace(
    ["cashatdoorstep"],
    "cod"
).replace(
    ["Easypay_MA", "easypay_voucher"],
    "Easypay"
).replace(
    ["internetbanking", "mygateway", "marketingexpense"],
    "Payaxis"
).replace(
    ["jazzvoucher"],
    "jazzwallet"
).replace(
    # Mark payment methods with less than 1000 transactions as "other"
    ["apg", "financesettlement", "productcredit", "mcblite", "ublcreditcard"],
    "other"
)
df["payment_method"].cat.categories.values

# A
#### Order status distribution

In [ ]:
sns.set()
status_dist = df.groupby("increment_id")["status"].first().value_counts()
status_dist.plot(kind="pie", autopct=lambda pct: "{:.1f}%\n({:d})".format(pct, int(pct / 100.*np.sum(status_dist))), colors=['lime', 'red', "steelblue"])

#### One-hot encode status category

In [ ]:
df = pd.concat([df.drop(columns=['status']), pd.get_dummies(df['status'], prefix='status')], axis=1)
df.columns.values

#### Calculate real order discount percentage (of grand total)


In [ ]:
df["item_total"] = df["price"] * df["quantity"]
orders_df = pd.concat([
    df.groupby("increment_id")["item_total"].sum(),
    df.groupby("increment_id")["grand_total"].first(),
    df.groupby("increment_id")["status_canceled"].first()
], axis=1).rename(columns={"item_total": "order_total"})
orders_df["real_discount_amount"] = (orders_df["order_total"]-orders_df["grand_total"]).clip(lower=0)
orders_df["discount"] = np.nan_to_num(orders_df["real_discount_amount"] / orders_df["order_total"], True, 0, 0).clip(max=1)
orders_df.head()

# B
#### Order and cancellation rate

In [ ]:
orders_df["discount (%)"] = np.floor(orders_df["discount"] * 100)
# orders_df["order_count"] = np.floor(orders_df["discount"] * 100)
order_bins_df_prime = orders_df[(orders_df["discount (%)"] <= 100) & (orders_df["discount (%)"] >= 0)]
order_bins_df = pd.concat([
    order_bins_df_prime.groupby("discount (%)").mean(),
    pd.DataFrame(order_bins_df_prime.groupby("discount (%)").size(), columns=["order_count"])
], axis=1)
order_bins_df["cancellation_rate (%)"] = order_bins_df["status_canceled"] * 100
order_bins_df.head()

In [ ]:
order_bins_line_df = order_bins_df.rolling(5).mean()
# sns.set(rc={"figure.figsize": (12, 6)})
o_fig, o_ax1 = plt.subplots(figsize=(12, 6))
sns.lineplot(order_bins_line_df, x=order_bins_line_df.index, y="cancellation_rate (%)", ax=o_ax1)
o_ax2 = o_ax1.twinx()
g = sns.lineplot(data=order_bins_line_df, x=order_bins_line_df.index, y='order_count', alpha=0.6, ax=o_ax2, c="orange")
g.set_yscale("log")

# C
#### Order total and cancellation rate (%)
with order count per bin

In [ ]:
otcdf = orders_df.copy()
otcdf["grand_total"] = np.floor(otcdf["grand_total"])
otcdf = otcdf[(otcdf["grand_total"] <= 10000000) & (otcdf["grand_total"] > 0)]
otc_bins_df = pd.concat([
    otcdf.groupby("grand_total")["status_canceled"].mean() * 100,
    pd.DataFrame(otcdf.groupby("grand_total").size(), columns=["order_count"]),
], axis=1).rename(columns={
    "status_canceled": "cancellation_rate (%)",
})
otc_bins_line_df = otc_bins_df.copy()
otc_bins_line_df["order_count"] = otc_bins_line_df["order_count"].rolling(200).mean()
otc_bins_line_df["cancellation_rate (%)"] = otc_bins_line_df["cancellation_rate (%)"].rolling(200).mean()
sns.set_style(style=None, rc=None)
otc_fig, otc_ax1 = plt.subplots(figsize=(12, 6))
g = sns.lineplot(data=otc_bins_line_df, x=otc_bins_line_df.index, y='cancellation_rate (%)', sort=False, ax=otc_ax1)
g.set_xscale("log")
otc_ax2 = otc_ax1.twinx()
sns.lineplot(data=otc_bins_line_df, x=otc_bins_line_df.index, y='order_count', alpha=0.6, ax=otc_ax2, c="orange")

# D
#### Order status distribution by payment method

In [ ]:
dfpm = df.drop_duplicates(subset=["increment_id"]).groupby("payment_method")[
    "status_completed", "status_canceled", "status_refunded"].sum()
dfpm.head()

In [ ]:
dfpm.plot(kind='bar', stacked=True, color=['lime', 'red', "steelblue"], figsize=(12, 6))

#### Mean number of items (sum of item type quantities) per order

In [ ]:
df.groupby("increment_id")["quantity"].sum().mean()

#### Mean number of item types

In [ ]:
df.groupby("increment_id").size().mean()

### Correlation between total quantities (item type quantities) and cancellation
Cancelled orders have very slightly higher total quantity, but basically the same as others

In [ ]:
pd.concat([
    df.groupby("increment_id")["status_canceled"].first(),
    df.groupby("increment_id")["quantity"].sum()
], axis=1).groupby("status_canceled")["quantity"].mean()

#### Mean number of unique item types by cancellation status
Cancelled orders are slightly smaller than non-canceled orders

In [ ]:
pd.concat([
    df.groupby("increment_id")["status_canceled"].first(),
    pd.DataFrame(df.groupby("increment_id").size(), columns=["order_size"])
], axis=1).groupby("status_canceled")["order_size"].mean()

#### Mean grand total by cancellation status
Cancelled orders have higher mean grand total than others

In [ ]:
pd.concat([
    pd.DataFrame(df.groupby("increment_id")["grand_total"].first(), columns=["grand_total"]),
    df.groupby("increment_id")["status_canceled"].first()
], axis=1).groupby("status_canceled")["grand_total"].mean()

# E
#### Cancellation rate and date created

In [ ]:
cac_df = df.drop_duplicates(subset=["increment_id"])
cac_df = pd.concat([
    cac_df.groupby("created_at")["status_canceled"].mean(),
    pd.DataFrame(cac_df.groupby("created_at").size(), columns=["order_count"])
], axis=1).rename(columns={"status_canceled": "cancellation_rate (%)"})
sns.set_style(style=None, rc=None)
cac_fig, cac_ax1 = plt.subplots(figsize=(16, 6))
cac_line_df = cac_df.copy()
# cac_line_df["cancellation_rate (%)"] = cac_line_df["cancellation_rate (%)"].rolling(5).mean()
cac_line_df = cac_line_df.rolling(14).mean()
sns.lineplot(data=cac_line_df, x=cac_line_df.index, y='cancellation_rate (%)', ax=cac_ax1)
cac_ax2 = cac_ax1.twinx()
sns.lineplot(data=cac_line_df, x=cac_line_df.index, y='order_count', alpha=0.8, ax=cac_ax2, c="orange")

#### Cancellation rate by customer age
Difficult to conclude, since order count drops quickly as customer age increases

In [ ]:
csc_df = df.drop_duplicates(subset=["increment_id"])
csc_df["customer_age"] = (csc_df["created_at"] - csc_df["customer_since"]).dt.days
# print(csc_df["customer_age"].describe())
csc_df = csc_df[csc_df["customer_age"] < 100]
csc_df = pd.concat([
    csc_df.groupby("customer_age")["status_canceled"].mean(),
    pd.DataFrame(csc_df.groupby("customer_age").size(), columns=["order_count"])
], axis=1).rename(columns={"status_canceled": "cancellation_rate (%)"})
sns.set_style(style=None, rc=None)
csc_fig, csc_ax1 = plt.subplots(figsize=(24, 6))
csc_line_df = csc_df.rolling(5).mean()
csc_g = sns.lineplot(data=csc_line_df, x=csc_line_df.index, y='cancellation_rate (%)', ax=csc_ax1)
csc_ax2 = csc_ax1.twinx()
sns.lineplot(data=csc_line_df, x=csc_line_df.index, y='order_count', alpha=0.6, ax=csc_ax2, c="orange")

# G
#### Cancellation rate and customer order number
Slight increase in cancellations as customer "order history" becomes longer

In [ ]:
csc2_df = df.drop_duplicates(subset=["increment_id"])
csc2_df["customer_nth_order"] = csc2_df.groupby("customer_id").cumcount()+1
csc2_df = csc2_df[(csc2_df["customer_nth_order"] > 0) & (csc2_df["customer_nth_order"] <= 10)]
csc2_df = pd.concat([
    csc2_df.groupby("customer_nth_order")["status_canceled"].mean(),
    pd.DataFrame(csc2_df.groupby("customer_nth_order").size(), columns=["order_count"])
], axis=1).rename(columns={"status_canceled": "cancellation_rate (%)"})
sns.set_style(style=None, rc=None)
csc2_fig, csc2_ax1 = plt.subplots(figsize=(10, 4))
# csc2_line_df = csc2_df.rolling(15).mean()
csc2_line_df =csc2_df
print(csc2_line_df)
sns.lineplot(data=csc2_line_df, x=csc2_line_df.index, y='cancellation_rate (%)', ax=csc2_ax1)
csc2_ax2 = csc2_ax1.twinx()
sns.lineplot(data=csc2_line_df, x=csc2_line_df.index, y='order_count', alpha=0.6, ax=csc2_ax2, c="orange")

#### Discount and cancellation status
Cancelled orders have a higher *absolute* discount. Tied to canceled orders having larger mean grand total

In [ ]:
orders_df.groupby("status_canceled")["real_discount_amount"].mean()

# F
#### Status distribution per category

In [ ]:
pd.concat([
    df.groupby("category")["status_completed", "status_canceled", "status_refunded", "quantity"].mean(),
    df.groupby("category")["quantity"].sum().rename("total_items")
], axis=1)
df.drop_duplicates(subset=["increment_id"]).groupby("category")[
    "status_completed", "status_canceled", "status_refunded"
].sum().plot(
    kind='bar', stacked=True,
    color=['lime', 'red', "steelblue"],
    figsize=(12, 6)
)